In [1]:
import os
import time

# # To generate GIFs
# !pip install imageio
# !pip install git+https://github.com/tensorflow/docs
    
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from IPython import display
import PIL
from PIL import Image

# Load and prepare the dataset

In [2]:
def prep_images(img_array):
    img_array = img_array.astype('float32')
    img_array = (img_array) / 255  # Normalize the images to [0, 1]
    return img_array

def rescale_image(img_array):
    img_array = np.array(img_array)
    img_array = img_array * 255
    img_array[img_array > 255] = 255
    img_array[img_array < 0] = 0
    return img_array.astype(int)

In [3]:
# Album Covers dataset:

general_path = r'C:\Users\weldl\Datasets\Album Covers Images'
def album_cover_load_data(folderpath=r'C:\Users\weldl\Datasets\Album Covers Images',
                          n_img_max=None, img_size=(128, 128)):
    imgs_dir = os.listdir(folderpath)
    if n_img_max is not None:
        imgs_dir = imgs_dir[0:n_img_max]
    train_images = np.array([np.asarray(Image.open(f'{general_path}/{filename}').resize(img_size)) for filename in imgs_dir])
    return train_images
    
train_images = album_cover_load_data(general_path, n_img_max=10000)
train_images = prep_images(train_images)
train_images.shape

MemoryError: Unable to allocate 1.83 GiB for an array with shape (10000, 128, 128, 3) and data type float32

In [ ]:
IMG_SHAPE = train_images.shape[1:]
BUFFER_SIZE = train_images.shape[0]
BATCH_SIZE = 256

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Create the models

In [ ]:
def make_generator_model_album_cover(input_shape=(100,)):
    model=tf.keras.Sequential()

    model.add(layers.Dense(4*4*512,input_shape=input_shape))
    model.add(layers.Reshape([4,4,512]))

    model.add(layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())
#     model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.BatchNormalization())

#     model.add(layers.Conv2DTranspose(16, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.BatchNormalization())

    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same",
                                     activation='sigmoid'))
    return model

generator = make_generator_model_album_cover()
generator.summary()

In [ ]:
# Use the (as yet untrained) generator to create an image.
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.figure()
plt.imshow(rescale_image(generated_image[0]))
# plt.imshow(generated_image[0, :, :, 0], cmap='gray')
plt.show()

In [ ]:
generated_image.shape

In [ ]:
# The Discriminator:
def make_discriminator_model(input_shape=(28, 28, 1)):
#     model=tf.keras.Sequential()
#     model.add(layers.Conv2D(32, kernel_size=4, strides=2, padding="same",input_shape = input_shape))
#     model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(0.2))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(0.2))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding="same"))
#     model.add(layers.LeakyReLU(0.2))
#     model.add(layers.Flatten())
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(1,activation='sigmoid'))

    model=tf.keras.Sequential()
    model.add(layers.Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

discriminator = make_discriminator_model(input_shape=IMG_SHAPE)
discriminator.summary()

In [ ]:
decision = discriminator(generated_image)
print (decision)

## Define the loss and optimizers

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop

In [ ]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
def train(dataset, epochs):
  epoch_array = []
  gen_loss_array = []
  disc_loss_array = []
  for epoch in range(epochs):
    start = time.time()
    
    epoch_gen_loss = []
    epoch_disc_loss = []
    for image_batch in dataset:
        gen_loss, disc_loss = train_step(image_batch)
        epoch_gen_loss.append(gen_loss)
        epoch_disc_loss.append(disc_loss)
    epoch_array.append(epoch + 1)
    gen_loss_array.append(np.mean(epoch_gen_loss))
    disc_loss_array.append(np.mean(epoch_disc_loss))

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    plot_gan_evolution(epoch_array, gen_loss_array, disc_loss_array)
    generate_and_save_images(generator, epoch + 1, seed)
    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  plot_gan_evolution(epoch_array, gen_loss_array, disc_loss_array)
  generate_and_save_images(generator,
                           epochs,
                           seed)



In [ ]:
def plot_gan_evolution(epoch_array, gen_loss_array, disc_loss_array, figsize=(10, 4)):
    plt.figure(figsize=figsize)
    plt.title("Models losses evolution")
    plt.plot(epoch_array, gen_loss_array, label='gen loss')
    plt.plot(epoch_array, disc_loss_array, label='disc loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(6, 6))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(rescale_image(predictions[i, :, :, :]))
#         plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('./imgs/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Create a GIF

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('./imgs/image_at_epoch_{:04d}.png'.format(epoch_no))

display_image(EPOCHS)

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('./imgs/image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)